In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import pprint
import requests
import time
nfl_key = 'my_api_key'

## Source

### https://developer.sportradar.com/

- nfl api

- scores from 2001 - 2017


## Get 10 years of scores form the NFL

- create a function:
    1. inputs year, season(reg or playoffs), api  key
    2. request data from api and returs json file

#### Create a loop to create a list for wanted years

In [2]:
# loop to create a list of years wanted
years = []
for i in range(2000,2018):
    years.append(str(i))
    
print(years)

['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']


In [3]:
# function
season = 'reg'
key = '5h567tf39qwsksfp9uxakxfc'
def get_scores(year,season, key):
    
    URL = 'https://api.sportradar.us/nfl-ot2/games/{}/{}/schedule.json?api_key={}'.format(year,season,key)
    r = requests.get(URL)
    return r.json()

## Loop over to get all the data wanted

In [4]:
appended_data = []  # emty list to append df
for i in years:
    score = get_scores(i,season,key)  # create json file using func
    time.sleep(1)
    for j in range(len(score['weeks'])):  # loop over every week given year
        data = json_normalize(score['weeks'][j]['games'])  # convert to df
        appended_data.append(data)  # append df
    nfl = pd.concat(appended_data, axis=0)   # concat all datframes from years given

In [5]:
nfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4593 entries, 0 to 15
Data columns (total 33 columns):
attendance             4592 non-null float64
away.alias             4593 non-null object
away.game_number       4593 non-null int64
away.id                4593 non-null object
away.name              4593 non-null object
broadcast.internet     81 non-null object
broadcast.network      1536 non-null object
broadcast.satellite    801 non-null object
entry_mode             4593 non-null object
home.alias             4593 non-null object
home.game_number       4593 non-null int64
home.id                4593 non-null object
home.name              4593 non-null object
id                     4593 non-null object
number                 1537 non-null float64
reference              1537 non-null object
scheduled              4593 non-null object
scoring.away_points    4592 non-null float64
scoring.home_points    4592 non-null float64
scoring.periods        4592 non-null object
status          

## Save this raw dataframe to csv

In [54]:
nfl.to_csv('clean_data/nfl10yearsfull.csv')

## Slice raw dataframe with wanted columns

In [7]:
nfl.head()

,attendance,away.alias,away.game_number,away.id,away.name,broadcast.internet,broadcast.network,broadcast.satellite,entry_mode,home.alias,...,venue.capacity,venue.city,venue.country,venue.id,venue.name,venue.roof_type,venue.state,venue.surface,venue.zip,weather
0,64900.0,DET,1,c5a59daa-53a7-4de0-851f-fb12be893e9e,Detroit Lions,NaN,NaN,NaN,INGEST,NO,...,76468,New Orleans,USA,3c85d89a-ec66-4983-acd5-1381d6c8673a,Mercedes-Benz Superdome,dome,LA,artificial,70112,"Partly Cloudly Temp: 89 F, Wind: West 10 mph"
1,60292.0,TB,1,4254d319-1bc7-4f81-b4ab-b5e6f3402b69,Tampa Bay Buccaneers,NaN,NaN,NaN,INGEST,NE,...,60292,Foxborough,NaN,67412aec-5fa4-4cef-ac07-323b90d9d388,Foxboro Stadium,outdoor,MA,turf,02035,"Cloudy Temp: 71 F, Humidity: 93%, Wind: Variab..."
2,72949.0,SEA,1,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seattle Seahawks,NaN,NaN,NaN,INGEST,MIA,...,76100,Miami Gardens,USA,50a5c833-1570-4c38-abc7-7914cf87dbde,Hard Rock Stadium,outdoor,FL,turf,33056,"Sunny Temp: 89 F, Humidity: 59%, Wind: East 13..."
3,59870.0,NYJ,1,5fee86ae-74ab-4bdd-8416-42a9dd9964f3,New York Jets,NaN,NaN,NaN,INGEST,GB,...,80750,Green Bay,USA,5a60dd3a-302c-41c6-ab0f-dd335c1103c2,Lambeau Field,outdoor,WI,turf,54304,"Mostly cloudy, light winds, high of 77. Temp: ..."
4,80257.0,CAR,1,f14bf5cc-9a82-4a38-bc15-d39f75ed5314,Carolina Panthers,NaN,NaN,NaN,INGEST,WAS,...,83000,Landover,USA,7c11bb2d-4a53-4842-b842-0f1c63ed78e9,FedExField,outdoor,MD,turf,20785,"Temp: 80 F, Humidity: 98%, Wind: Calm mph"


In [8]:
nfl10 = nfl[['scheduled','home.alias','scoring.home_points','away.alias','scoring.away_points']]

In [9]:
nfl10.head()

,scheduled,home.alias,scoring.home_points,away.alias,scoring.away_points
0,2000-09-03T18:03:36+00:00,NO,10.0,DET,14.0
1,2000-09-03T18:04:35+00:00,NE,16.0,TB,21.0
2,2000-09-03T21:15:54+00:00,MIA,23.0,SEA,0.0
3,2000-09-03T21:20:12+00:00,GB,16.0,NYJ,20.0
4,2000-09-03T19:02:58+00:00,WAS,20.0,CAR,17.0


## scheduled column

- Convert to datetime
- set as index
- sort index

In [13]:
nfl10['scheduled'] = pd.to_datetime(nfl10['scheduled'])
nfl10 = nfl10.set_index('scheduled').sort_index(ascending=True)

/home/salas/miniconda3/envs/springboard/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
nfl10.head()

,home.alias,scoring.home_points,away.alias,scoring.away_points
scheduled,,,,
2000-09-03 00:00:00,PIT,0.0,BAL,16.0
2000-09-03 00:00:00,OAK,9.0,SD,6.0
2000-09-03 18:02:00,ATL,36.0,SF,28.0
2000-09-03 18:03:33,NYG,21.0,ARI,16.0
2000-09-03 18:03:36,NO,10.0,DET,14.0


## Winner Column

Create a column where scores are compared and the largest is the winner

In [15]:
nfl10['winner'] = np.where(nfl10['scoring.home_points'] > nfl10['scoring.away_points'], nfl10['home.alias'], nfl10['away.alias'])

In [16]:
nfl10.head()

,home.alias,scoring.home_points,away.alias,scoring.away_points,winner
scheduled,,,,,
2000-09-03 00:00:00,PIT,0.0,BAL,16.0,BAL
2000-09-03 00:00:00,OAK,9.0,SD,6.0,OAK
2000-09-03 18:02:00,ATL,36.0,SF,28.0,ATL
2000-09-03 18:03:33,NYG,21.0,ARI,16.0,NYG
2000-09-03 18:03:36,NO,10.0,DET,14.0,DET


## Inspect Data

In [21]:
nfl10.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4593 entries, 2000-09-03 00:00:00 to 2017-12-31 21:25:00
Data columns (total 5 columns):
home.alias             4593 non-null object
scoring.home_points    4592 non-null float64
away.alias             4593 non-null object
scoring.away_points    4592 non-null float64
winner                 4593 non-null object
dtypes: float64(2), object(3)
memory usage: 375.3+ KB


In [18]:
nfl10['home.alias'].unique()

array(['PIT', 'OAK', 'ATL', 'NYG', 'NO', 'NE', 'KC', 'WAS', 'MIN', 'DAL',
       'MIA', 'GB', 'BUF', 'CLE', 'STL', 'IND', 'CIN', 'BAL', 'TB', 'PHI',
       'SEA', 'DEN', 'DET', 'SF', 'SD', 'ARI', 'TEN', 'NYJ', 'JAC', 'CAR',
       'CHI', 'HOU', 'LA', 'LAC'], dtype=object)

In [20]:
nfl10['home.alias'].value_counts(dropna=False)

MIA    145
IND    144
CAR    144
BAL    144
NO     144
ARI    144
NYG    144
CIN    144
SF     144
DAL    144
PHI    144
CLE    144
TB     144
PIT    144
BUF    144
DET    144
JAC    144
SEA    144
ATL    144
GB     144
NE     144
NYJ    144
WAS    144
MIN    144
CHI    144
OAK    144
TEN    144
DEN    144
KC     144
SD     136
HOU    128
STL    128
LA      16
LAC      8
Name: home.alias, dtype: int64

In [22]:
nfl10['scoring.home_points'].unique()  # has one nan value

array([ 0.,  9., 36., 21., 10., 16., 14., 20., 30., 23.,  7., 41., 31.,
       27., 39., 18., 13., 34., 42., 15., 22., 32., 17.,  6., 24., 19.,
       37.,  3., 12., 43., 57., 28., 38., 26., 29., 45., 44., 48., 54.,
       40., 49.,  8., 35., 52., 25., 33., 47., 50., 56., 58., 51.,  5.,
       11., 59., 62., 46., 55.,  2., 53., nan])

In [23]:
nfl10['scoring.home_points'].unique()  # has one nan value

 20.0    339
 24.0    302
 17.0    285
 27.0    281
 31.0    244
 13.0    221
 10.0    209
 23.0    206
 21.0    173
 34.0    167
 16.0    157
 14.0    150
 30.0    150
 28.0    147
 7.0     121
 26.0    112
 38.0    106
 19.0    106
 6.0      80
 35.0     80
 22.0     76
 3.0      75
 37.0     62
 9.0      59
 41.0     52
 33.0     51
 0.0      48
 29.0     48
 25.0     42
 12.0     42
 45.0     41
 15.0     41
 18.0     34
 42.0     34
 32.0     27
 48.0     27
 44.0     27
 36.0     26
 40.0     21
 49.0     19
 39.0     16
 43.0     15
 52.0     12
 51.0     10
 11.0      9
 47.0      9
 8.0       7
 56.0      4
 55.0      4
 46.0      4
 54.0      3
 58.0      2
 59.0      2
 57.0      2
 5.0       1
NaN        1
 53.0      1
 50.0      1
 62.0      1
 2.0       1
Name: scoring.home_points, dtype: int64

In [24]:
nfl10['away.alias'].unique()  # ok

array(['BAL', 'SD', 'SF', 'ARI', 'DET', 'TB', 'IND', 'CAR', 'CHI', 'PHI',
       'SEA', 'NYJ', 'TEN', 'JAC', 'DEN', 'OAK', 'CLE', 'GB', 'NYG',
       'MIA', 'STL', 'ATL', 'WAS', 'NO', 'DAL', 'KC', 'NE', 'BUF', 'CIN',
       'PIT', 'MIN', 'HOU', 'LA', 'LAC'], dtype=object)

In [25]:
nfl10['away.alias'].value_counts(dropna=False)

TB     145
DET    144
IND    144
CAR    144
BAL    144
NO     144
ARI    144
NYG    144
CIN    144
SF     144
DAL    144
PHI    144
CLE    144
PIT    144
BUF    144
MIA    144
JAC    144
SEA    144
ATL    144
GB     144
NE     144
NYJ    144
WAS    144
MIN    144
CHI    144
OAK    144
TEN    144
DEN    144
KC     144
SD     136
HOU    128
STL    128
LA      16
LAC      8
Name: away.alias, dtype: int64

In [26]:
nfl10['scoring.away_points'].unique()  # has one nan value

array([16.,  6., 28., 14., 21., 27., 17., 41.,  0., 20., 13., 36., 38.,
       24., 18., 33.,  7., 37., 10., 31., 19.,  3., 15., 23., 29., 12.,
       34.,  9., 35., 30., 11., 26., 48., 45., 22., 25., 42., 32., 40.,
       44., 49., 43., 55., 39.,  5.,  8., 56., 54., 47., 46., 59., 52.,
       51., 50., nan])

In [28]:
nfl10['scoring.away_points'].value_counts(dropna=False) # has one nan value

 17.0    351
 24.0    304
 20.0    300
 10.0    288
 13.0    266
 27.0    234
 14.0    229
 23.0    219
 7.0     197
 21.0    173
 31.0    164
 16.0    156
 3.0     139
 34.0    132
 28.0    119
 30.0    109
 6.0     103
 26.0     95
 38.0     93
 19.0     93
 0.0      83
 9.0      69
 33.0     66
 35.0     60
 37.0     59
 41.0     54
 22.0     52
 25.0     44
 15.0     43
 29.0     42
 12.0     41
 18.0     32
 42.0     25
 36.0     24
 45.0     20
 32.0     18
 40.0     18
 44.0     12
 48.0     11
 11.0     10
 49.0     10
 8.0       7
 39.0      7
 43.0      5
 51.0      3
 47.0      2
 5.0       2
 59.0      2
 56.0      2
 55.0      1
 52.0      1
NaN        1
 54.0      1
 46.0      1
 50.0      1
Name: scoring.away_points, dtype: int64

## Find nan value

- create mask to find nan value

In [43]:
nfl10[nfl10['scoring.home_points'].isnull()]

,home.alias,scoring.home_points,away.alias,scoring.away_points,winner
scheduled,,,,,
2017-09-10 17:00:00,MIA,NaN,TB,NaN,TB


## Google it!

- it was posponed

[google search](https://www.google.com/search?q=2017-09-10+miami+vs+tampa+bay&rlz=1C1CHBD_enUS769US769&oq=2017-09-10++miami+vs+tampa+bay&aqs=chrome..69i57.12899j0j4&sourceid=chrome&ie=UTF-8#sie=m;/g/11f53rgny4;6;/m/059yj;dt;fp;1)

## Save clean version to csv

In [53]:
nfl10.to_csv('clean_data/nfl10clean.csv')

## Select only games form 2008 - 2017

In [50]:
nfl_08_17 = nfl10.loc['2008':'2017']

In [51]:
nfl_08_17.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2561 entries, 2008-09-04 23:07:39 to 2017-12-31 21:25:00
Data columns (total 5 columns):
home.alias             2561 non-null object
scoring.home_points    2560 non-null float64
away.alias             2561 non-null object
scoring.away_points    2560 non-null float64
winner                 2561 non-null object
dtypes: float64(2), object(3)
memory usage: 120.0+ KB


## Save clean  selected data to  to csv

In [52]:
nfl_08_17.to_csv('clean_data/nfl_08_17_clean.csv')

In [55]:
ls clean_data/

astros_08_17_clean.csv*       mls_clean.csv*        rice_08_17_clean.csv*
dynamo_08_17_clean.csv*       nfl_08_17_clean.csv*  tsu_08_17_clean.csv*
hou_rockets_08_17_clean.csv*  nfl10clean.csv*       uh_08_17_clean.csv*
mls_08_16_clean.csv*          nfl10yearsfull.csv*


In [56]:
## Finish